In [22]:
import os
from bs4 import BeautifulSoup
from playwright.async_api import async_playwright, TimeoutError as PlaywrightTimeout
import time

In [23]:
#Getting seasons 2021-2022 and 2022-2023
SEASONS = [2022,2023]

In [24]:
#Creating pointers to where we want our data to go
DATA_DIR = 'data1'
STANDINGS_DIR = os.path.join(DATA_DIR, "standings1")
SCORES_DIR = os.path.join(DATA_DIR, "scores1")

In [25]:
#Function to get html for us
#async because playwright needs to run async in notebook

async def get_html(url, selector, sleep = 5, retries = 3):
    html = None
    for i in range(1, retries+1):
        # pauses our program for i amount of seconds to not scrape too fast
        time.sleep(sleep * i)
        
        #try block of code to handle errors
        try:
            async with async_playwright() as p:
                #opens chromium browser
                #await pauses execution of code until async operation is done
                browser = await p.firefox.launch()
                #new page in browser with our specific URL
                page = await browser.new_page()
                await page.goto(url)
                #print url for progress check
                print(await page.title())
                #we grab our specific piece of HTML from website
                html = await page.inner_html(selector)
        #if there is issue w/ scraping, we know there is a timeout and then retries i times
        except PlaywrightTimeout:
            print(f"Timeout Error on {url}")
            continue
        #if there are no errors, then don't retry
        else:
            break
    return html

In [26]:
#function to get links for all seasons
async def scrape_season(season):
    url = f"https://www.basketball-reference.com/leagues/NBA_{season}_games.html"
    html = await get_html(url, "#content .filter")
    
    soup = BeautifulSoup(html)
    #get all links with anchor tags from html "a"
    links = soup.find_all("a")
    #getting all links with href within it
    href = [l["href"] for l in links]
    #creating full links for standing pages
    standing_pages = [f"https://basketball-reference.com{l}" for l in href]
    
    #saving to disk so we dont process while scraping
    for url in standing_pages:
        save_path = os.path.join(STANDINGS_DIR, url.split('/')[-1])
        #if we already scraped, don't scrape again
        if os.path.exists(save_path):
            continue
        
        html = await get_html(url, '#all_schedule')
        with open(save_path, 'w+') as f:
            f.write(html)  

In [27]:
#loop through each season and get pages
for season in SEASONS:
    await scrape_season(season)

2021-22 NBA Schedule | Basketball-Reference.com
Timeout Error on https://basketball-reference.com/leagues/NBA_2022_games-november.html
2021-22 NBA Schedule | Basketball-Reference.com
2021-22 NBA Schedule | Basketball-Reference.com
Timeout Error on https://basketball-reference.com/leagues/NBA_2022_games-january.html
2021-22 NBA Schedule | Basketball-Reference.com
2021-22 NBA Schedule | Basketball-Reference.com
2021-22 NBA Schedule | Basketball-Reference.com
2021-22 NBA Schedule | Basketball-Reference.com
2021-22 NBA Schedule | Basketball-Reference.com
2021-22 NBA Schedule | Basketball-Reference.com
2022-23 NBA Schedule | Basketball-Reference.com
2022-23 NBA Schedule | Basketball-Reference.com
2022-23 NBA Schedule | Basketball-Reference.com
2022-23 NBA Schedule | Basketball-Reference.com
2022-23 NBA Schedule | Basketball-Reference.com
2022-23 NBA Schedule | Basketball-Reference.com
2022-23 NBA Schedule | Basketball-Reference.com
2022-23 NBA Schedule | Basketball-Reference.com
2022-23 NBA

In [28]:
#checking if I have all the files in the directory
standings_files = os.listdir(STANDINGS_DIR)

In [29]:
async def scrape_game(standings_file):
    with open(standings_file, 'r') as f:
        html = f.read()

    #cleaning links
    soup = BeautifulSoup(html)
    links = soup.find_all('a')
    hrefs = [l.get('href') for l in links]
    boxscores = [l for l in hrefs if l and 'boxscore' in l and '.html' in l]
    boxscores = [f"https://basketball-reference.com{l}" for l in boxscores]

    for url in boxscores:
        save_path = os.path.join(SCORES_DIR, url.split('/')[-1])
        if os.path.exists(save_path):
            continue

        html = await get_html(url, '#content')
        if not html:
            continue
        with open(save_path, 'w+') as f:
            f.write(html)

In [ ]:
for f in standings_files:
    filepath = os.path.join(STANDINGS_DIR, f)
    await scrape_game(filepath)

Nets vs Bucks, October 19, 2021 | Basketball-Reference.com
Warriors vs Lakers, October 19, 2021 | Basketball-Reference.com
Pacers vs Hornets, October 20, 2021 | Basketball-Reference.com
Bulls vs Pistons, October 20, 2021 | Basketball-Reference.com
Celtics vs Knicks, October 20, 2021 | Basketball-Reference.com
Wizards vs Raptors, October 20, 2021 | Basketball-Reference.com
Timeout Error on https://basketball-reference.com/boxscores/202110200MEM.html
Cavaliers vs Grizzlies, October 20, 2021 | Basketball-Reference.com
Rockets vs Timberwolves, October 20, 2021 | Basketball-Reference.com
76ers vs Pelicans, October 20, 2021 | Basketball-Reference.com
Magic vs Spurs, October 20, 2021 | Basketball-Reference.com
Thunder vs Jazz, October 20, 2021 | Basketball-Reference.com
Kings vs Trail Blazers, October 20, 2021 | Basketball-Reference.com
Nuggets vs Suns, October 20, 2021 | Basketball-Reference.com
Mavericks vs Hawks, October 21, 2021 | Basketball-Reference.com
Timeout Error on https://basketba